# CRISPRCasdb
CRISPRCasdb is a database of CRISPR arrays and cas loci identified in GenBank's whole-genome assemblies using CRISPRCasFinder. The following tables were downloaded from the [version dated 21 Jan 2021](https://crisprcas.i2bc.paris-saclay.fr/Home/Download):
1. `crisprdata_4.csv`: evidence level 4 (highest-confidence) CRISPR arrays
2. `spacerdata_4.csv`: spacer sequences of evidence level 4 CRISPR arrays
3. `drdata_4.csv`: direct repeat (dr) sequences of evidence level 4 CRISPR arrays
4. `casdata.csv`: cas loci
5. `casgenedata.csv`: cas genes
6. `seqdata.csv`: sequence information of all strains, including those without CRISPR/cas
7. `taxa.csv`: all taxa

with the following columns:

| crisprdata_4         | spacerdata_4 | drdata_4    | casdata      | casgenedata      | seqdata     | taxa           |
|----------------------|--------------|-------------|--------------|------------------|-------------|----------------|
| genbank              | genbank      | genbank     | genbank      | genbank          | taxid       | id             |
| seqid                | seqid        | seqid       | seqid        | seqid            | genbank     | parent         |
| locus_start          | locus_start  | locus_start | locus_start  | locus_start      | refseq      | scientificname |
| locus_length         | start        | start       | locus_length | gene             | seqid       | rank           |
| orientation          | sequence     | sequence    | subtype      | gene_start       | category    |                |
| drconsensus          |              |             |              | gene_length      | seq_length  |                |
| drconservation       |              |             |              | gene_orientation | ncount      |                |
| spacerconservation   |              |             |              |                  | description |                |
| potentialorientation |              |             |              |                  |             |                |
| evidencelevelreeval  |              |             |              |                  |             |                |
| blastscore           |              |             |              |                  |             |                |

The rows in all tables are ordered (if applicable) by: (a) taxid, (b) genbank, (c) seq_length (descending), (d) locus_start, and (e) spacer/dr/casgene start.

## Step 1: Parse CRISPRCasdb tables

In [1]:
using DelimitedFiles

include("get_distrib_sortedarr.jl");

### CRISPRCasFinder evidence levels of CRISPR arrays to use

In [2]:
el = "4" # evidence level 4
# el = "34" # evidence levels 3–4
# el = "234" # evidence levels 2–4
# el = "1234" # evidence levels 1–4;

### genbank &rightarrow; crisprloci = [genbank, seqid, locus_start]

In [3]:
crisprdata, crisprheader = readdlm("CRISPRCasdb/2021/crisprdata_$el.csv",',',header=true)

crisprloci = [crisprdata[i,1:3] for i in 1:size(crisprdata,1)] # [genbank, seqid, locus_start]
# crisprinfo = [crisprdata[i,:] for i in 1:size(crisprdata,1)] # [genbank, seqid, locus_start, ...]

gs, cum = get_cum_sortedarr(crisprdata[:,1]) # cumulative frequencies of genbanks in crisprdata

genbank2crisprloci = Dict{String,Array}(gs[i] => crisprloci[cum[i]+1:cum[i+1]] for i in 1:length(gs))
# genbank2crisprinfo = Dict{String,Array}(gs[i] => crisprinfo[cum[i]+1:cum[i+1]] for i in 1:length(gs));

### crisprlocus &rightarrow; spacers

In [4]:
spacerdata, spacerheader = readdlm("CRISPRCasdb/2021/spacerdata_$el.csv",',',header=true)

crisprlocus_ofspacers = [spacerdata[i,1:3] for i in 1:size(spacerdata,1)] # [genbank, seqid, locus_start]
ls, cum = get_cum_sortedarr(crisprlocus_ofspacers) # cumulative frequencies of crisprloci in spacerdata

# crisprlocus2spacerstarts = Dict{Array,Array{Int}}(ls[i] => spacerdata[cum[i]+1:cum[i+1],4] for i in 1:length(ls))
crisprlocus2spacers = Dict{Array,Array{String}}(ls[i] => spacerdata[cum[i]+1:cum[i+1],5] for i in 1:length(ls));

### crisprlocus &rightarrow; drs

In [5]:
drdata, drheader = readdlm("CRISPRCasdb/2021/drdata_$el.csv",',',header=true)

crisprlocus_ofdrs = [drdata[i,1:3] for i in 1:size(drdata,1)] # [genbank, seqid, locus_start]
ls, cum = get_cum_sortedarr(crisprlocus_ofdrs) # cumulative frequencies of crisprloci in drdata

# crisprlocus2drstarts = Dict{Array,Array{Int}}(ls[i] => drdata[cum[i]+1:cum[i+1],4] for i in 1:length(ls))
crisprlocus2drs = Dict{Array,Array{String}}(ls[i] => drdata[cum[i]+1:cum[i+1],5] for i in 1:length(ls));

### genbank &rightarrow; casloci = [genbank, seqid, locus_start]

In [6]:
casdata, casheader = readdlm("CRISPRCasdb/2021/casdata.csv",',',header=true)

casloci = [casdata[i,1:3] for i in 1:size(casdata,1)] # [genbank, seqid, locus_start]

gs, cum = get_cum_sortedarr(casdata[:,1]) # cumulative frequencies of genbanks in casdata

genbank2casloci = Dict{String,Array}(gs[i] => casloci[cum[i]+1:cum[i+1]] for i in 1:length(gs));

### caslocus &rightarrow; casgene = [gene, gene_start, gene_length, gene_orientation]

In [7]:
casgenedata, casgeneheader = readdlm("CRISPRCasdb/2021/casgenedata.csv",',',header=true)

casgenes = [casgenedata[i,4:end] for i in 1:size(casgenedata,1)]

casloci_incasgenes = [casgenedata[i,1:3] for i in 1:size(casgenedata,1)] # [genbank, seqid, locus_start]
cl, cum = get_cum_sortedarr(casloci_incasgenes) # cumulative frequencies of cas loci in casgenedata

caslocus2genes = Dict{Array,Array}(cl[i] => casgenes[cum[i]+1:cum[i+1]] for i in 1:length(cl));

In [8]:
casgenenames = [[cg[1] for cg in cgs] for cgs in values(caslocus2genes)];

### taxid &rightarrow; parentid/name/rank

In [9]:
taxa, taxaheader = readdlm("CRISPRCasdb/2021/taxa.csv",',',header=true)

taxid2parentid = Dict{Int,Union{Int,AbstractString}}(taxa[i,1] => taxa[i,2] for i in 1:size(taxa,1))
taxid2name = Dict{Int,String}(taxa[i,1] => taxa[i,3] for i in 1:size(taxa,1))
taxid2rank = Dict{Int,String}(taxa[i,1] => taxa[i,4] for i in 1:size(taxa,1));

### genbank &rightarrow; taxid, taxid &rightarrow; genbanks
Note: This includes strains without CRISPR/cas.

In [10]:
seqdata, seqheader = readdlm("CRISPRCasdb/2021/seqdata.csv",',',header=true)

genbank2taxid = Dict{String,Int}(seqdata[i,2] => seqdata[i,1] for i in 1:size(seqdata,1))

ids, cum = get_cum_sortedarr(seqdata[:,1]) # cumulative frequencies of taxids in seqdata

taxid2genbanks = Dict{Int,Array{String}}(ids[i] => unique(seqdata[cum[i]+1:cum[i+1],2]) for i in 1:length(ids));

### genbank &rightarrow; seqinfo = [seqid, category, seq_length, ncount, description], genbank &rightarrow; L, genbank &rightarrow; refseq
"seqid" is a UUID given by CRISPRCasdb unique to each sequence.  
"category": 1 = circular, 2 = linear  
"description" says whether sequence is chromosome or plasmid (we do not make a distinction in our analysis).  
L is sum of all seq_length of a strain.

In [11]:
gs, cum = get_cum_sortedarr(seqdata[:,2]) # cumulative frequencies of genbanks in seqdata

seqinfo = [seqdata[i,4:8] for i in 1:size(seqdata,1)] # [seqid, category, seq_length, ncount, description]

genbank2seqinfo = Dict{String,Array}(gs[i] => seqinfo[cum[i]+1:cum[i+1]] for i in 1:length(gs))
genbank2L = Dict{String,Int}(g => sum([s[3] for s in genbank2seqinfo[g]]) for g in gs)
genbank2refseq = Dict{String,String}(gs[i] => seqdata[cum[i]+1,3] for i in 1:length(gs));

## Step 2: Define cas subtype variables
### caslocus &rightarrow; subtype
"CAS" = subtype not given

In [12]:
caslocus2subtype = Dict{Array,String}(casloci[i] => casdata[i,5] for i in 1:size(casdata,1));

### genbank &rightarrow; subtype
"M" = strain contains multiple subtypes of cas loci  
See `analysis-by-subtype.ipynb` for the 12 subtypes we analyze.

In [13]:
genbank2subtype = Dict{String,String}()

for g in keys(genbank2casloci)
    sts = [caslocus2subtype[c] for c in genbank2casloci[g]]
    if length(unique(sts)) == 1
        genbank2subtype[g] = sts[1]
    else
        genbank2subtype[g] = "M" # multiple subtypes
    end
end

In [14]:
subtypes_bytype = [["CAS-TypeI"*a for a in ["A","B","C","D","E","F","U"]], # type I
                   ["CAS-TypeII"*a for a in ["A","B","C"]], # type II
                   ["CAS-TypeIII"*a for a in ["A","B","C","D"]], # type III
                   ["CAS-TypeIV"], # type IV
                   ["CAS-TypeV"*a for a in ["A","B"]], # type V
                   ["CAS-TypeVI"*a for a in ["A","B1","B2","C"]]] # type VI

# for labeling plots
subtypes_bytype1 = [["I-"*a for a in ["A","B","C","D","E","F","U"]], # type I
                    ["II-"*a for a in ["A","B","C"]], # type II
                    ["III-"*a for a in ["A","B","C","D"]], # type III
                    ["IV"], # type IV
                    ["V-"*a for a in ["A","B"]], # type V
                    ["VI-"*a for a in ["A","B1","B2","C"]]] # type VI

subtypes_all = vcat(["M"],["CAS"],subtypes_bytype...);

## Step 3: Group strains (including those without CRISPR/cas) by species
### taxid &rightarrow; speciesid/genusid

In [15]:
taxid2speciesid = Dict{Int,Int}()

for t0 in keys(taxid2genbanks)
    t = t0
    while taxid2rank[t] != "species"
        t = taxid2parentid[t]
        if taxid2rank[t] == "genus"
            println("Warning: taxid $t0 has no species-level parent. Using genus instead.")
            break
        end
    end
    taxid2speciesid[t0] = t
end

In [16]:
# # print phylogeny of taxon t
# t = 1235284
# println(taxid2name[t],"\t",taxid2rank[t])
# while taxid2rank[t] != "superkingdom"
#     t = taxid2parentid[t]
#     println(taxid2name[t],"\t",taxid2rank[t])
# end

In [17]:
# taxid2genusid = Dict{Int,Int}()

# # because not all strains have an assigned genus
# ranks = ["genus","family","order","class","phylum","superkingdom"]

# for t0 in keys(taxid2genbanks)
#     t = t0
#     while true
#         t = taxid2parentid[t]
#         ind = findfirst(x->x==taxid2rank[t],ranks)
#         if ind != nothing
#             taxid2genusid[t0] = t
#             if ind > 1
#                 println("Warning: taxid $t0 has no genus-level parent. Using $(taxid2rank[t]) instead.")
#             end
#             break
#         end
#     end
# end

In [18]:
# ##################################################
# # uncomment to do genus-level filtering in `analysis-across-species.ipynb`
# ##################################################
# taxid2speciesid = taxid2genusid;

### speciesid &rightarrow; genbanks/kingdom

In [19]:
speciesid2genbanks = Dict{Int,Array{String}}()

for (t,gs) in taxid2genbanks
    s = taxid2speciesid[t]
    if haskey(speciesid2genbanks,s)
        speciesid2genbanks[s] = vcat(speciesid2genbanks[s],gs)
    else
        speciesid2genbanks[s] = copy(gs)
    end
end

# sort genbanks of each speciesid
for s in keys(speciesid2genbanks)
    speciesid2genbanks[s] = sort(speciesid2genbanks[s])
end

In [20]:
speciesid2kingdom = Dict{Int,String}()

for t0 in keys(speciesid2genbanks)
    t = t0
    while taxid2rank[t] != "superkingdom"
        t = taxid2parentid[t]
    end
    speciesid2kingdom[t0] = taxid2name[t]
end

## Step 4: Group strains by presence/absense of CRISPR and cas
### genbank &rightarrow; cid, speciesid &rightarrow; cids
cid: 0 = CRISPR-cas-, 1 = CRISPR+cas-, 2 = CRISPR-cas+, and 3 = CRISPR+cas+.

In [21]:
genbank2cid = Dict{String,Int}()

for (t,gs) in taxid2genbanks
    for g in gs
        cid = 0
        if haskey(genbank2crisprloci,g)
            cid += 1
        end
        if haskey(genbank2casloci,g)
            cid += 2
        end
        genbank2cid[g] = cid
    end
end

In [22]:
speciesid2cids = Dict{Int,Array{Int}}(s => [genbank2cid[g] for g in gs] for (s,gs) in speciesid2genbanks);

## Step 5: Compile CRISPR+cas+, CRISPR+cas-, CRISPR-cas+, and CRISPR+ strains by species
### speciesid &rightarrow; genbanks_cc/cnc/ncc/c
"cc" = CRISPR+cas+, "cnc" = CRISPR+cas-, "ncc" = CRISPR-cas+, and "c" = CRISPR+.

In [23]:
speciesid2genbanks_cc = Dict{Int,Array{String}}()
speciesid2genbanks_cnc = Dict{Int,Array{String}}()
speciesid2genbanks_c = Dict{Int,Array{String}}()
speciesid2genbanks_ncc = Dict{Int,Array{String}}()

for (s,cids) in speciesid2cids
    # CRISPR+cas+
    inds = findall(x->x==3,cids)
    if length(inds) > 0
        speciesid2genbanks_cc[s] = speciesid2genbanks[s][inds]
    end
    # CRISPR+cas-
    inds = findall(x->x==1,cids)
    if length(inds) > 0
        speciesid2genbanks_cnc[s] = speciesid2genbanks[s][inds]
    end
    # CRISPR+
    inds = findall(x->x==1||x==3,cids)
    if length(inds) > 0
        speciesid2genbanks_c[s] = speciesid2genbanks[s][inds]
    end
    # CRISPR-cas+
    inds = findall(x->x==2,cids)
    if length(inds) > 0
        speciesid2genbanks_ncc[s] = speciesid2genbanks[s][inds]
    end
end